In [21]:
# note, you need to add parquet to flink installation, follow instruction as per class session
# you must take new Dockerfile on day2 github folder, then build image, then run the cluster

# demonstration of using same code for stream and batch

In [ ]:
import os
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment

from pyflink.table import EnvironmentSettings, TableEnvironment

import get_env
# for stream
# env = get_env.get_remote_env()
# t_env = StreamTableEnvironment.create(env)

# for batch
env, t_env = get_env.get_remote_batch_env()

conf = t_env.get_config().get_configuration()


# === Python Exec Location ===
conf.set_string("python.executable", "/usr/bin/python3")
conf.set_string("pipeline.jars", "file:///opt/flink/plugins/gs-fs-hadoop/flink-gs-fs-hadoop-1.20.2.jar")  # client-side path

# === Allow fallback to Hadoop FS for gs:// and s3:// ===
conf.set_string("fs.allowed-fallback-filesystems", "hadoop")

# GCS (Hadoop connector) - optional if already present in flink-conf.yaml
# conf.set_string("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
# conf.set_string("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# conf.set_string("google.cloud.auth.service.account.enable", "true")
# conf.set_string("google.cloud.auth.service.account.json.keyfile", "/etc/gcp/key.json")


t_env.get_config().set("parallelism.default", "1")

In [ ]:
conf = t_env.get_config().get_configuration()
conf.set_string("fs.allowed-fallback-filesystems", "hadoop")
conf.set_string("fs.gs.project.id", "flink-demo-470113")
conf.set_string("fs.gs.auth.service.account.json.keyfile", "/etc/gcp/key.json")

In [ ]:
t_env.execute_sql("DROP TABLE IF EXISTS ratings_parquet")

t_env.execute_sql("""
CREATE TABLE ratings_parquet (
  `userId` INT,
  `movieId` INT,
  `rating` DOUBLE,
  `timestamp` BIGINT
) WITH (
  'connector' = 'filesystem',
  'path' = 'gs://gk2-datalake/silver/ratings/',
  'format' = 'parquet'
)
""")



In [ ]:
from pyflink.table.expressions import col

ratings = t_env.from_path("ratings_parquet")

# comment out, taken only 100 records for demo
# ratings = ratings.limit (100)

# group_by using Table attributes (these are Expression objects)
ratings_agg = (
    ratings
    .group_by(ratings.movieId)
    .select(
        ratings.movieId.alias('r_movieId'),
        ratings.userId.count.alias("user_count"),
        ratings.rating.avg.alias("avg_rating")
    )
    .filter((col("user_count") > 100) & (col("avg_rating") >= 4.0))
    # or 
    # .filter(col("user_count") > 100)
    # .filter(col("avg_rating") >= 4.0)
)

# ratings_agg.limit(5).execute().print()   # RUNS ON REMOTE JOBMANAGER

In [ ]:
t_env.execute_sql("DROP TABLE IF EXISTS movies_parquet").wait()

ddl_parquet = """
CREATE TABLE movies_parquet (
    movieId INT,
    title   STRING,
    genres  STRING,
    PRIMARY KEY (`movieId`) NOT ENFORCED
) WITH (
    'connector' = 'filesystem',
    'path'      = 'gs://gk2-datalake/silver/movies/',
    'format'    = 'parquet'
)
"""
t_env.execute_sql(ddl_parquet).wait()


In [ ]:
# get table handles
movies  = t_env.from_path("movies_parquet")

# 2) join on renamed column
popular_movies = (
    ratings_agg
    .join(movies)
    .where(ratings_agg.r_movieId == movies.movieId)
    .select(
        movies.movieId,        # keep canonical movieId from movies table
        movies.title,
        movies.genres,
        ratings_agg.user_count,
        ratings_agg.avg_rating
    )
)

t_env.execute_sql("DROP TEMPORARY VIEW IF EXISTS popular_movies_temp")

    
t_env.create_temporary_view("popular_movies_temp", popular_movies)

# 4) show / write
# joined.limit(5).execute().print()

result = t_env.sql_query("SELECT * FROM popular_movies_temp ORDER BY avg_rating LIMIT 10 ")

result.execute().print()

In [18]:
# 1) Create the Iceberg catalog 'movielens' (Hadoop-style catalog storing metadata in the warehouse path)
# Adjust 'warehouse' to your desired Iceberg metadata location on GCS.

t_env.execute_sql("""    
CREATE CATALOG IF NOT EXISTS movielens WITH (
  'type' = 'iceberg',
  'catalog-type' = 'hadoop',             -- use 'hive' or 'rest' / 'glue' if you run those catalogs
  'warehouse' = 'gs://gks-datalake/iceberg-warehouse/'
);
""")


# 2) Tell Flink to use the new catalog

# t_env.execute_sql("""    
# USE CATALOG movielens;
# """)

# 3) Create the database (schema) 'gold' if it doesn't exist
t_env.execute_sql("""   
CREATE DATABASE IF NOT EXISTS movielens.gold;
""")

# 4) Drop the table if it already exists (safe)

t_env.execute_sql("""   
DROP TABLE IF EXISTS movielens.gold.popular_movies;
""")

# 5) Create the Iceberg table in movielens.gold.popular_movies
t_env.execute_sql("""  

CREATE TABLE movielens.gold.popular_movies (
  movieId     INT,
  title       STRING,
  genres      STRING,
  user_count  BIGINT,
  avg_rating  DOUBLE,
  PRIMARY KEY (movieId) NOT ENFORCED
) WITH (
 -- 'connector' = 'iceberg',
   -- 'write.format.default' = 'parquet',
  -- 'engine.hive.enabled' = 'false',
  'format-version' = '2'                 -- recommended: Iceberg table format v2
  -- other iceberg table options may go here, e.g. write.format.default, etc.
);
""")


In [19]:
# async submit to remote JobManager
# t_env.get_config().get_configuration().set_string("execution.runtime-mode", "BATCH")

popular_movies.execute_insert("movielens.gold.popular_movies").wait()

In [ ]:

result = t_env.sql_query("SELECT * FROM movielens.gold.popular_movies LIMIT 10 ")

result.execute().print()

In [20]:
t_env.execute_sql("DROP TEMPORARY TABLE IF EXISTS popular_movies")

t_env.execute_sql("""
CREATE TEMPORARY TABLE popular_movies (
    movieId     INT,
    title       STRING,
    genres      STRING,
    user_count  BIGINT,
    avg_rating  DOUBLE,
    PRIMARY KEY (movieId) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url'       = 'jdbc:mysql://mysql:3306/ecomm',
  'table-name'= 'popular_movies',
  'username'  = 'team',
  'password'  = 'team1234'
)
""")

popular_movies.execute_insert("popular_movies").wait()

# t_env.execute_sql("TRUNCATE TABLE popular_movies")
# t_env.execute_sql("INSERT INTO popular_movies SELECT * FROM popular_movies_temp")
